In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic-english/ara-eng (1)/_about.txt
/kaggle/input/arabic-english/ara-eng (1)/ara.txt


In [2]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, GRU

2025-04-21 22:09:03.758525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745273343.781100      98 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745273343.788104      98 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
input_texts=[]
target_texts=[]
datapath='/kaggle/input/arabic-english/ara-eng (1)/ara.txt'

with open (datapath,"r",encoding="Utf_8")as f:
    lines =f.read().split("\n")

In [4]:
for line in lines[:min(100000, len(lines) - 1)]:
    try:
        input_text, target_text, _ = line.split('\t')
        target_text = "start" + target_text + "end"

        input_texts.append(input_text)
        target_texts.append(target_text)

    except ValueError:
        print(f"error in {line}")
        continue

In [5]:
input_tokenizer = Tokenizer(filters="", lower=True, oov_token="<OOV>")
target_tokenizer = Tokenizer(filters="", lower=True, oov_token="<OOV>")

#fit on text
input_tokenizer.fit_on_texts(input_texts)
target_tokenizer.fit_on_texts(target_texts)

# conert text to sequense
input_sequences = input_tokenizer.texts_to_sequences(input_texts)
target_sequences = target_tokenizer.texts_to_sequences(target_texts)

#get vocablary sizes
num_encoder_tokens = len(input_tokenizer.word_index) +1      # for padding
num_decoder_tokens = len(target_tokenizer.word_index) +1

In [6]:
print ( 'I Vocabulary Size (Input):',num_encoder_tokens)
print ( 'I Vocabulary Size (Target):',num_decoder_tokens)

I Vocabulary Size (Input): 6868
I Vocabulary Size (Target): 17120


In [7]:
# max length
max_encoder_seq_length = max(len(seq) for seq in input_sequences)
max_decoder_seq_length = max(len(seq) for seq in target_sequences)

#paddeing
encoder_input_data = pad_sequences(input_sequences, maxlen = max_encoder_seq_length, padding="post")
decoder_input_data = pad_sequences(target_sequences, maxlen = max_decoder_seq_length, padding="post")

decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:,:-1] = decoder_input_data[:,1:]
decoder_target_data[:,-1] = 0

print ("max Sequense Length (input):" ,max_encoder_seq_length)
print ("max Sequense Length (Target):" ,max_decoder_seq_length)

max Sequense Length (input): 34
max Sequense Length (Target): 36


In [8]:
encoder_input_data

array([[1938,    0,    0, ...,    0,    0,    0],
       [3700,    0,    0, ...,    0,    0,    0],
       [1939,    0,    0, ...,    0,    0,    0],
       ...,
       [   6,  211, 2950, ...,    0,    0,    0],
       [2594,   41, 6855, ...,    0,    0,    0],
       [  62,   20, 6862, ..., 3614, 1525, 6867]], dtype=int32)

In [9]:
latent_dim = 128 # num of neron
embedding_dim = 100 

#encoder
encoder_inputs = Input(shape=(max_encoder_seq_length,))
encoder_embeding = Embedding(num_encoder_tokens, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embeding)
encoder_states= [state_h, state_c]

#Decoder
Decoder_inputs = Input(shape=(max_decoder_seq_length,))
Decoder_embeding = Embedding(num_decoder_tokens, embedding_dim, mask_zero=True)(Decoder_inputs)
Decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
Decoder_outputs, _, _ = Decoder_lstm(Decoder_embeding, initial_state = encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation = "softmax")
Decoder_outputs = decoder_dense(Decoder_outputs)

model = Model([encoder_inputs, Decoder_inputs],Decoder_outputs)
model.compile(optimizer="adam", loss = "sparse_categorical_crossentropy")

I0000 00:00:1745273347.519583      98 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745273347.520244      98 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [10]:
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 34)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 36)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 34, 100)        │        686,800 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, 34)             │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 36, 100)        │      1,712,000 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 128), (None,   │        117,248 │ embedding[0][0],       │
│                           │ 128), (None, 128)]     │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 36, 128),      │        117,248 │ embedding_1[0][0],     │
│                           │ (None, 128), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 128)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 36, 17120)      │      2,208,480 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,841,776 (18.47 MB)

 Trainable params: 4,841,776 (18.47 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=16,
          epochs=50,
          validation_split=0.2,
          callbacks=[early_stopping])

Epoch 1/50
627/627 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 5.0531 - val_loss: 9.3535
Epoch 2/50
627/627 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 4.6895 - val_loss: 10.0952
Epoch 3/50
627/627 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 4.3084 - val_loss: 10.1565
Epoch 4/50
627/627 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 3.9151 - val_loss: 10.7147
Epoch 5/50
627/627 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 3.5124 - val_loss: 10.9321
Epoch 6/50
627/627 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 3.1670 - val_loss: 11.1748
